In [200]:
#簡略型ファジィ推論を示す
#簡略型とTSKの違いは後件部を実数値にするか、一次結合にするかの違い
import numpy as np
import pandas as pd
import random
import math
import sys

In [201]:
#まず医療診断データを読み取る
diabetes_data = pd.read_csv('fuzzy_diabetes2_1.csv')

In [202]:
#教師データの数は73、テストデータに数は72と設定する
TEACHER_SIZE = 73
TEST_SIZE = 72

In [203]:
#教師データとテストデータに分割する
teacher_data = diabetes_data.iloc[0:TEACHER_SIZE]
test_data = diabetes_data.iloc[TEACHER_SIZE:]

In [204]:
#説明変数と被説明変数に分ける
x_teacher_data = np.array(teacher_data[['input1','input2','input3','input4','input5']])
y_teacher_data = np.array(teacher_data['output'])
x_test_data = np.array(test_data[['input1','input2','input3','input4','input5']])
y_test_data = np.array(test_data['output'])

In [205]:
#ファジィルールの数と、ファジィ分割数、入力変数の数を設定する
NUMBER_OF_FUZZY_PARTITION = 3
NUMBER_OF_INPUT = 5
NUMBER_OF_FUZZY_RULE = NUMBER_OF_FUZZY_PARTITION ** NUMBER_OF_INPUT

In [206]:
#前件部の初期値と後件部の初期値を設定する
#前件部の中心と幅は、ルール数×各ルールにおける入力変数の数
antecedent_center = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
antecedent_broad = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
consequent = np.empty(NUMBER_OF_FUZZY_RULE)

#後件部の値は0.3に初期設定する
for i in range(NUMBER_OF_FUZZY_RULE):
    consequent[i] = 0.3
#前件部を初期設定する
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(NUMBER_OF_INPUT):
        if ((int)(i / (NUMBER_OF_FUZZY_PARTITION ** (NUMBER_OF_INPUT - (j+1))))) % NUMBER_OF_FUZZY_PARTITION == 0:
            antecedent_center[i,j] = 0
            antecedent_broad[i,j] = 1
        elif ((int)(i / (NUMBER_OF_FUZZY_PARTITION ** (NUMBER_OF_INPUT - (j+1))))) % NUMBER_OF_FUZZY_PARTITION == 1:
            antecedent_center[i,j] = 0.5
            antecedent_broad[i,j] = 0.5
        else:
            antecedent_center[i,j] = 1
            antecedent_broad[i,j] = 1

In [207]:
#次に学習を行う
#学習回数を設定する
TRAIN_TIME = 500
#学習係数を設定する
LEARNING_ANTECEDENT_BROAD = 0.001
LEARNING_ANTECEDENT_CENTER = 0.001
LEARNING_CONSEQUENT = 0.001
#まず、各ルールに基づいて入力変数をファジィ化（メンバシップ関数に代入）する
membership_function = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))

for time in range(TRAIN_TIME):
    for i in range(TEACHER_SIZE):
        delta_broad = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
        delta_center = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
        #適合度を初期化する
        adaptability = np.ones(NUMBER_OF_FUZZY_RULE)
        for j in range(NUMBER_OF_FUZZY_RULE):
            for k in range(NUMBER_OF_INPUT):
                if(x_teacher_data[i,k] >= antecedent_center[j,k] - antecedent_broad[j,k]) and x_teacher_data[i,k] <= antecedent_center[j,k]:
                    membership_function[j,k] = (x_teacher_data[i,k] - (antecedent_center[j,k] - antecedent_broad[j,k])) / antecedent_broad[j,k]
                    delta_broad[j,k] = (antecedent_center[j,k] - x_teacher_data[i,k]) / (antecedent_broad[j,k]**2)
                    delta_center[j,k] = -(1 / antecedent_broad[j,k])
                elif(x_teacher_data[i,k] > antecedent_center[j,k]) and (x_teacher_data[i,k] <= antecedent_center[j,k] + antecedent_broad[j,k]):
                    membership_function[j,k] = -(x_teacher_data[i,k] - (antecedent_center[j,k] + antecedent_broad[j,k])) / antecedent_broad[j,k]
                    delta_broad[j,k] = -(antecedent_center[j,k] - x_teacher_data[i,k]) / (antecedent_broad[j,k]**2)
                    delta_center[j,k] = 1 / antecedent_broad[j,k]
                #else:
                    #print('**')
            #各ルールにおける適合度を求める
            for k in range(NUMBER_OF_INPUT):
                adaptability[j] = adaptability[j] * membership_function[j,k]
        #各データに対して予測結果を求める
        output = 0
        output = np.sum(np.dot(adaptability, consequent)) / np.sum(adaptability)
        if(time == TRAIN_TIME-1):
            print(output, y_teacher_data[i])
        #出力値を求めた次に後件部と前件部の更新を行う
        for j in range(NUMBER_OF_FUZZY_RULE):
            consequent[j] = consequent[j] + (LEARNING_CONSEQUENT * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output)
            for k in range(NUMBER_OF_INPUT):
                antecedent_center[j,k] = antecedent_center[j,k] + (LEARNING_ANTECEDENT_CENTER * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output) * (consequent[j] - output) * delta_center[j,k]
                antecedent_broad[j,k] = antecedent_broad[j,k] + (LEARNING_ANTECEDENT_BROAD * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output) * (consequent[j] - output) * delta_broad[j,k]

0.4492986325664817 0.5
0.6370011582441484 1.0
0.3498708421008675 0.0
0.5127599154999637 0.5
0.7546017174759084 1.0
0.35282598063864073 0.0
0.6398302112075148 1.0
0.5061028413098995 0.0
0.4823819517616047 1.0
0.7459587420689061 1.0
0.545307489603948 1.0
0.6806998093528751 1.0
0.7240247307274571 1.0
0.4471723444980054 0.5
0.5453671208063541 1.0
0.6809598182488652 1.0
0.4471808180572401 0.0
0.731621113871176 1.0
0.5180784262707309 0.5
0.8543230770764846 1.0
0.660608777827815 1.0
0.7638188566872794 1.0
0.4886905768441911 1.0
0.6352297873416034 1.0
0.7208284613353224 1.0
0.2702430168681054 0.0
0.614582108451931 1.0
0.6366710790487036 1.0
0.5883844876731305 0.5
0.25855423154744417 0.0
0.4885410799311159 0.5
0.32725201375631036 0.0
0.4200224467904648 0.5
0.7717455328258687 1.0
0.3885132150432898 0.5
0.48255539904342565 1.0
0.6121386584971166 0.5
0.388624655420876 0.0
0.3598812160029542 0.0
0.5263762280408201 1.0
0.25855253591908833 0.0
0.42279950108658615 0.5
0.47352454248386017 0.5
0.6374386

In [209]:
#次にテストデータを用いて結果を確認する
#各ルールに基づいてテストデータの入力変数をファジィ化する
test_membership_function = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))

for i in range(TEST_SIZE):
    #適合度を初期化する
    test_adaptability = np.ones(NUMBER_OF_FUZZY_RULE)
    for j in range(NUMBER_OF_FUZZY_RULE):
        for k in range(NUMBER_OF_INPUT):
            if(x_test_data[i,k] >= antecedent_center[j,k] - antecedent_broad[j,k]) and x_test_data[i,k] <= antecedent_center[j,k]:
                test_membership_function[j,k] = (x_test_data[i,k] - ( antecedent_center[j,k] - antecedent_broad[j,k])) / antecedent_broad[j,k]
            elif(x_test_data[i,k] > antecedent_center[j,k]) and (x_test_data[i,k] <= antecedent_center[j,k] + antecedent_broad[j,k]):
                test_membership_function[j,k] = -(x_test_data[i,k] - (antecedent_center[j,k] + antecedent_broad[j,k])) / antecedent_broad[j,k]
        #各ルールにおける適合度を求める
        for k in range(NUMBER_OF_INPUT):
            test_adaptability[j] = test_adaptability[j] * test_membership_function[j,k]
    #各データに対して予測結果を求める
    test_output = 0
    test_output = np.sum(np.dot(test_adaptability, consequent)) / np.sum(test_adaptability)
    print(test_output, y_test_data[i])

0.42879228296531474 0.5
0.48267801002690985 1.0
0.29281688604475764 0.0
0.6738211128776584 1.0
0.6895249538609814 1.0
0.6123172170416359 0.5
0.7387373414343027 1.0
0.3272549058742332 0.0
0.2596882642133744 0.0
0.6376572663609434 1.0
0.4200983646142606 0.5
0.6821219424296313 1.0
0.7661963885153091 1.0
0.415927598334983 0.5
0.28389519744765246 0.0
0.6623565914992141 1.0
0.3272549058742332 0.0
0.595186588994513 0.5
0.6386349026256323 1.0
0.8236881168594 1.0
0.4888726023701823 1.0
0.595186588994513 0.5
0.3226783069534347 0.0
0.636959073256663 1.0
0.5584668015979151 0.5
0.5456905537768763 1.0
0.7535621748374132 1.0
0.4200983646142606 0.5
0.732911015062513 1.0
0.5660953859050245 1.0
0.38866814941877104 0.0
0.7321850490672904 1.0
0.28175852879756264 0.0
0.6123172170416359 0.5
0.5064084717349597 0.0
0.6630120876126617 1.0
0.2754208759701235 0.0
0.3272549058742332 0.0
0.4495550141732064 0.5
0.31919800572566126 0.0
0.6274829165889634 1.0
0.6386349026256323 1.0
0.7871479981509336 1.0
0.7553592793